<a href="https://colab.research.google.com/github/BrunaMoraisMenezesDSM/fatec_PLN_Codes/blob/master/Aula%208%20-%20Introdu%C3%A7%C3%A3o%20a%20Machine%20Learning%20para%20PLN/%5BPLN%5DAula_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 08 - Introdução a Machine Learning para PLN

## Exemplo 01 - Aplicação do modelo de Naives em um texto

In [8]:
# Passo 1 - Criando o Corpus

corpus = [
    ("Eu amo PLN", "positivo"),
    ("Eu odeio bugs", "negativo"),
    ("Amo resolver problemas", "positivo"),
    ("Odeio erros", "negativo"),
    ("Bugs são legais", "positivo"),
    ("Resolver dificuldades é vida", "positivo"),
    ("Eu não gosto de falhas", "negativo"),
]
  # Atribuindo textos e a sua classificação, sendo positivo ou negativo

# Passo 2 - Processando o Texto
import re # Importando o módulo re para manipulação de strings com expressões regulares
from collections import defaultdict, Counter # Importando defaultdict e Counter para contagem de palavras e classes

# Função que processa o texto, convertendo para minúsculas e extraindo as palavras
def preprocess_text(text):
  return re.findall(r'\b\w+\b', text.lower()) # Encontrando palavras no texto em minúsculo com uma expressão regular

# Processando o corpus e associando cada texto à sua classe
processed_corpus = [(preprocess_text(text), label) for text, label in corpus] # Aplicando o pré-processamento a cada texto
print(processed_corpus) # Exibindo o corpus processado, mostrando as palavras e suas classes

# Passo 3 - Calculando probabilidades
class_counts = Counter() # Contagem de classes (positivo, negativo)
word_counts = defaultdict(Counter) # Contagem das palavras para cada classe
total_words = defaultdict(int) # Contagem total de palavras para cada classe

# Contagem das palavras por classe
for words, label in processed_corpus:
  class_counts[label] += 1 # Conta a ocorrência de cada classe
  for word in words:
    word_counts[label][word] += 1 # Conta a ocorrência de cada palavra em sua classe
    total_words[label] # Conta o total de palavras em cada classe

# Calculando as probabilidades de uma classe ser escolhida
total_examples = sum(class_counts.values()) # Total de textos
prior_probabilities = {cls: count / total_examples for cls, count in class_counts.items()} # Probabilidade de cada classe
  # prior_probabilities -> Dicionário que armazena a probabilidade a priori de cada classe
  # count ->  Quantidade de vezes que a classe 'cls' aparece no corpus
  # total_examples -> Número total de textos

# Função que calcula a probabilidade condicional de uma palavra dada uma classe
def conditional_probability(word, label, alpha=1):
  # word -> Palavra que estamos verificando
  # label -> Classe para a qual estamos calculando a probabilidade condicional
  # alpha -> Parâmetro de suavização de Laplace. O valor padrão é 1

  # Fórmula de Laplace para suavização de probabilidades
  return (word_counts[label][word] + alpha) / (total_words[label] + alpha * len(word_counts[label])) # Fórmula de Laplace
    # word_counts[label][word] -> Quantidade de vezes a palavra apareceu na classe 'label'
    # total_words[label] -> Número total de palavras na classe 'label'
    # alpha * len(word_counts[label]) -> Número total de possíveis palavras que podem aparecer na classe, multiplicado por 'alpha' para garantir que a probabilidade nunca seja zero

# Passo 4 - Classificar um novo texto
def predict(text):
  words = preprocess_text(text) # Processando o texto de entrada
  probabilities = {} # Armazena as probabilidades de cada classe

# Calculando as probabilidades para cada classe
  for label in class_counts.keys(): # Iterando sobre todas as classes presentes no conjunto de dados
    probabilities[label] = prior_probabilities[label] # Inicializando com a probabilidade escolhida da classe
    # A probabilidade a priori é a chance de um texto pertencer a uma determinada classe sem considerar as palavras específicas

    for word in words: # Iterando sobre todas as palavras do texto processado
      probabilities[label] *= conditional_probability(word, label) # Multiplicando a probabilidade condicional de cada palavra
  return max(probabilities, key=probabilities.get), probabilities # Retornando a classe com a maior probabilidade
  # key=probabilities.get ->  faz com que a função max escolha classe com o valor de probabilidade mais alto

# Passo 5 - Teste com um novo texto
novo_texto = "Eu amo resolver bugs" # Texto a ser classificado
classe, probs = predict(novo_texto) # Antecipa a classe do novo texto

# Exibindo o texto de entrada, classe prevista e probabilidades de cada classe
print(f'Texto: "{novo_texto}"')
print(f'Classe prevista: "{classe}"')
print(f'Probabilidades:')
for label, prob in probs.items(): # Iterando classes e exibindo suas probabilidades
  print(f'{label}"{prob}"')

[(['eu', 'amo', 'pln'], 'positivo'), (['eu', 'odeio', 'bugs'], 'negativo'), (['amo', 'resolver', 'problemas'], 'positivo'), (['odeio', 'erros'], 'negativo'), (['bugs', 'são', 'legais'], 'positivo'), (['resolver', 'dificuldades', 'é', 'vida'], 'positivo'), (['eu', 'não', 'gosto', 'de', 'falhas'], 'negativo')]
Texto: "Eu amo resolver bugs"
Classe prevista: "positivo"
Probabilidades:
positivo"0.0014050562510367166"
negativo"0.0006277901785714285"


## Exemplo 02 - Classificação do SVM (Support Vector Machines)

In [9]:
!pip install scikit-learn # Instalando biblioteca é usada para aprendizado de máquina, a scikit-learn

Modelo do SVM

In [10]:
# Passo 1 - Importando as bibliotecas a serem utilizadas
from sklearn.feature_extraction.text import TfidfVectorizer # Importando o vetorizador TF-IDF para transformar texto em vetores
from sklearn.svm import SVC # Importando o modelo Support Vector Classifier (SVM)
from sklearn.model_selection import train_test_split # Importando a função para dividir o dataset em treino e teste
from sklearn.metrics import classification_report # Importando a função para gerar relatório de avaliação

# Passo 2 - Dados exemplo
# Criando textos de exemplo
corpus = [
    "Eu amo PLN", "Eu odeio bugs", "Eu amo resolver problemas",
    "Odeio erros", "Amo programação", "Não gosto de falhas"
]
classes = ["positivo", "negativo", "positivo", "negativo", "positivo", "negativo"] # Classes correspondentes aos textos acima

# Passo 3 - Pré-processamento e vetorização
vectorizer = TfidfVectorizer() # Criando o objeto que aplica a transformação TF-IDF
X = vectorizer.fit_transform(corpus) # Convertendo o corpus em uma matriz esparsa de características
  # X -> São os dados de entrada
y = classes # Definindo as classes correspondentes
  # y -> São os rótulos ou classes

# Passo 4 - Dividindo dos dados e treinamento do modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
  # O método divide aleatoriamente os dados em dois conjuntos
    # X_train -> Conjunto de dados de treinamento
    # X_test -> Conjunto de dados de teste
    # y_train -> Labels de treinamento correspondentes aos dados em X_train
    # y_test -> Labels de teste correspondentes aos dados em X_test
  # test_size = 0.3 -> Define a proporção de dados que será alocada para o conjunto de teste
  # random_state = 42 -> Define a semente do gerador de números aleatórios

svm_model = SVC(kernel='linear') # Criando o modelo SVM com kernel linear
svm_model.fit(X_train, y_train) # Treinando o modelo com os dados de treino

# Passo 5 - Avaliação do modelo
y_pred = svm_model.predict(X_test) # Fazendo previsões usando o modelo
print(classification_report(y_test, y_pred)) # Exibindo o relatório de avaliação do modelo

              precision    recall  f1-score   support

    negativo       1.00      1.00      1.00         1
    positivo       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



comparação dos dois modelos

## Exemplo 03 - Comparando os classificadores com Scikit-Learn

In [11]:
# 1. Importando Bibliotecas
import nltk # Importando o pacote NLTK para processamento de linguagem natural
from sklearn.feature_extraction.text import TfidfVectorizer # Importando o vetorizador TF-IDF
from sklearn.model_selection import train_test_split # Importando a função para dividir o dataset
from sklearn.naive_bayes import MultinomialNB # Importando o classificador Naive Bayes para multinomial
from sklearn.svm import SVC # Importando o classificador Support Vector Machine (SVM)
from sklearn.metrics import classification_report # Importando a função para relatório de avaliação

# Baixando o dataset de exemplo
nltk.download('movie_reviews') # Baixando o conjunto de dados "movie_reviews" do NLTK
from nltk.corpus import movie_reviews # Importando o corpus de críticas de filmes do NLTK

# 2. Preparando dos dados
# Coletando textos e classes
documents = [(" ".join(movie_reviews.words(fileid)), category) # Criando uma lista com o texto e a categoria de cada crítica
    for category in movie_reviews.categories() # Iterando sobre as categorias de crítica (positivo, negativo)
    for fileid in movie_reviews.fileids(category)] # Iterando sobre os arquivos de cada categoria

# Separando textos e rótulos
texts, labels = zip(*documents) # Separando os textos e as classes em duas listas

# Transformando rótulos (positivo/negativo) em 0 e 1
from sklearn.preprocessing import LabelEncoder # Importando o codificador de rótulos
label_encoder = LabelEncoder() # Criando o codificador
labels = label_encoder.fit_transform(labels) # Transformando as classes em valores numéricos (0 ou 1)

 # Dividindo dados em treino e teste
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.3, random_state=42)
  # O método divide aleatoriamente os dados em dois conjuntos
    # X_train -> Conjunto de dados de treinamento
    # X_test -> Conjunto de dados de teste
    # y_train -> Labels de treinamento correspondentes aos dados em X_train
    # y_test -> Labels de teste correspondentes aos dados em X_test
  # test_size = 0.3 -> Define a proporção de dados que será alocada para o conjunto de teste
  # random_state = 42 -> Define a semente do gerador de números aleatórios

# 3. Representando do texto com TF-IDF
# Criando o vetorizador TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Criando o vetor TF-IDF, limitando o número de palavras para as 5000 mais frequentes

# Ajustando e transformando textos
X_train = vectorizer.fit_transform(texts_train) # Ajustando o vetor e transforma os textos de treino
X_test = vectorizer.transform(texts_test) # Transformando os textos de teste

# 4. Treinar os modelos
# Treinamento do Naive Bayes
nb_model = MultinomialNB() # Criando o modelo Naive Bayes para dados multinomiais
nb_model.fit(X_train, labels_train) # Treinando o modelo com os dados de treino

# Predição
nb_predictions = nb_model.predict(X_test) # Fazendo previsões usando o modelo Naive Bayes

# Treinamento do SVM
svm_model = SVC(kernel='linear') # Criando o modelo SVM com kernel linear
svm_model.fit(X_train, labels_train) # Treinando o modelo SVM com os dados de treino

  # Predição
svm_predictions = svm_model.predict(X_test) # Fazendo previsões usando o modelo SVM

# 5. Avaliação

# Avaliação do Naive Bayes
print("Naive Bayes Performance:")
print(classification_report(labels_test, nb_predictions, target_names=label_encoder.classes_))
  # Exibindo as métricas de desempenho para o modelo Naive Bayes
    # classification_report -> Gera uma tabela de métricas de avaliação para cada classe
    # labels_test -> Classes verdadeiras de cada amostra no conjunto de teste
    # nb_predictions -> Previsões feitas pelo modelo Naive Bayes para o conjunto de teste
    # target_names=label_encoder.classes_ -> Nomes das classes para as quais as métricas de desempenho serão calculadas
    # label_encoder.classes_ -> Rótulos codificados numericamente

# Avaliação do SVM
print("SVM Performance:")
print(classification_report(labels_test, svm_predictions, target_names=label_encoder.classes_))
  # Exibindo as métricas de performance para o modelo SVM
    # classification_report -> Gera uma tabela de métricas de avaliação para cada classe
    # labels_test -> Classes verdadeiras de cada amostra no conjunto de teste
    # svm_predictions -> Previsões feitas pelo modelo SVM para o conjunto de teste
    # target_names=label_encoder.classes_ -> Nomes das classes para as quais as métricas de desempenho serão calculadas
    # label_encoder.classes_ -> Rótulos codificados numericamente

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Naive Bayes Performance:
              precision    recall  f1-score   support

         neg       0.79      0.84      0.81       302
         pos       0.82      0.77      0.80       298

    accuracy                           0.80       600
   macro avg       0.80      0.80      0.80       600
weighted avg       0.80      0.80      0.80       600

SVM Performance:
              precision    recall  f1-score   support

         neg       0.82      0.80      0.81       302
         pos       0.81      0.82      0.81       298

    accuracy                           0.81       600
   macro avg       0.81      0.81      0.81       600
weighted avg       0.81      0.81      0.81       600

